In [1]:
import QuantLib as ql
import pandas as pd
import numpy as np
import os 
import quantFin as qf
import pymongo as py
import mysql.connector
import scipy.optimize as sp
import math
import matplotlib.pyplot as plt
import time
import datetime as dt
import sys

In [2]:
config = {
    'user': 'root',
    'password': 'cba123',
    'host': '34.66.65.69',
    'database':'ACT_01',
    'use_pure':True,

}
cnxn = mysql.connector.connect(**config)
cursor = cnxn.cursor()

In [3]:
#get mortality assumptions and store them in a dictionary for quick referencing
cursor.execute("select * from mortality_assumptions")
out=cursor.fetchall()
out

wareHouse=qf.AssumptionWareHouse()
for o in out:
    mortality=qf.MortalityAssumption()
    mortality.id=o[0]
    mortality.soa_table_name=o[1]
    mortality.gamma_makeham=o[2]
    mortality.beta_makeham=o[3]
    mortality.alpha_makeham=o[4]
    mortality.gender=o[5]
    wareHouse.mort_dept[mortality.id]=mortality
    

In [4]:
cnxn.close()



In [4]:
cell=qf.Cell()
cell.wareHouse.assump_set_dept[1]=qf.AssumptionSet(id=1,mortality_id=1)
cell.wareHouse.assump_set_dept[2]=qf.AssumptionSet(id=2,mortality_id=2)

for o in out:
    mortality=qf.MortalityAssumption()
    mortality.id=o[0]
    mortality.soa_table_name=o[1]
    mortality.gamma_makeham=o[2]
    mortality.beta_makeham=o[3]
    mortality.alpha_makeham=o[4]
    mortality.gender=o[5]
    cell.wareHouse.mort_dept[mortality.id]=mortality

In [5]:
ass_set=qf.AssumptionSet(id=1,mortality_id=1)
act_bucket=qf.ActuarialBucket(id=1,assumption_set_key=1)

wareHouse.assump_set_dept[ass_set.id]=ass_set
cell.mitos.append(act_bucket)
policy=qf.policyHolder()
policy.age=35
policy.assumption_set_id=1



In [6]:
start_time = time.time()
l=[]
for n in range(1,1000):
    l.append(qf.policyHolder())
n=0
for p in l:
    p.assumption_set_id=1
    p.age=min(35 +0.25*n,85)
    n=n+1
for p in l:
    cell.mitos[0].calculate_qx(p,cell.wareHouse)    
print(start_time-time.time())



-0.04760885238647461


In [7]:
policyX=qf.policyHolder()
policyX.assumption_set_id=1
policyX.age=40

In [8]:
intRate=qf.IntRatesStruct()
dateTime=qf.DateTimeStruct()
myFlatRate=0.035
discount_curve=qf.create_discount_curve(cell.mitos[0].valuationDate,myFlatRate,intRate)


In [12]:
bc=qf.ActuarialBucket()
#z=ql.SimpleCashFlow(ss[0],bc.valuationDate)

In [13]:
class IntX:
    def __init__(self,valDate):
        self.intRate=qf.IntRatesStruct()
        self.dateTime=qf.DateTimeStruct()


    def create_cashflows(self,startTime,freq,values):
        """
        DESC:
        creates cash flows based on a numpy array of values a starting date and a frequency 
        which increment the startTime in fixed steps
        INPUTS:
        startTime:Date=the starting valuation date ql.date
        freq:str= an string defining the metric of time 1->annual  12->monthly
        values:numpy array
        
        OUTPUT:
        a list of SimpleCashFlow objects
        """
        cashflows=[]
        for value in values:
            elapsedTime=np.where(values == value)[0][0] +1 #first index is always 0
            per=ql.Period(int(12*freq*elapsedTime),ql.Months) #must be multiplied by 12 because of freq basis
            futureTime=ql.Date.endOfMonth(startTime+per)
            cf=ql.SimpleCashFlow(value,futureTime)
            cashflows.append(cf)
        return cashflows

def calculate_sx_lx_with_lapse(qx,lapse_x):
    
    lxWithLapse=np.zeros(qx.size)
    sxWithLapse=np.zeros(qx.size)
    
    sxWithLapse[0]=1
    lxWithLapse[0]=qx[0]
    for index in range(1,qx.size):
        
        sxWithLapse[index]=sxWithLapse[index-1]*(1-qx[index-1])*(1-lapse_x[index-1])
        lxWithLapse[index]=sxWithLapse[index]*qx[index]
    return sxWithLapse,lxWithLapse

In [14]:
ss=IntX(bc.valuationDate)

In [15]:
cfs=[ql.SimpleCashFlow(100,ql.Date(31,10,1931)),ql.SimpleCashFlow(100,ql.Date(31,10,1932))]

result=ql.CashFlows.npv(cfs,discount_curve,True,bc.valuationDate)
result

183.200089792709

In [20]:
zz=cell.mitos[0].calculate_dx(policyX,cell.wareHouse)
startDate=ql.Date(31,10,1929)
aa=qf.create_cashflows(0,startDate,1,zz*10000)
for a in aa:
    print(a.amount())


22.236639608319564
23.48173418388111
24.844812750296484
26.33653900157573
27.968449422466346
29.753005441056636
31.70364428765504
33.83482724235609
36.162083617287124
38.70204841881386
41.4724911606168
44.492332740773136
47.78164664438922
51.3616399781236
55.254608974932836
59.48386261905747
64.0736069287573
69.04878119823654
74.43483614887447
80.25744249253397
86.5421168997027
93.31375084704385
100.59602637251808
108.41070150672482
116.77674723532871
125.70931749187442
135.21853416214813
145.30807075705198
155.97352172531777
167.2005498728546
178.96281267089975
191.2196801017607
203.913772903469
216.9683714460835
230.28477274959934
243.73970688900903
257.18296440097487
270.435432847419
283.28779197887457
295.50017017605734
306.8031154001627
316.9002748129812
325.473198899085
332.1886758124054
336.70894461392487
338.70501517624996
337.8731211171426
333.95403693819156
326.7545965414437
316.1702668804717
302.2070887146669
285.0007558223183
264.8301569308398
242.1224746690001
217.44706670

In [18]:
cursor.execute("select * from spot_rates where id=3")
out=cursor.fetchall()


In [20]:
bc.valuationDate

Date(27,10,1929)

In [27]:
cfs=[ql.SimpleCashFlow(100,ql.Date(31,10,1931)),ql.SimpleCashFlow(100,ql.Date(31,10,1932))]
spotValues=list(out[0][4:50])
startDate=ql.Date(31,10,1929)
dcurve=qf.create_spotCurve(spotValues,startDate,datetimeStruct2,intRate)


In [22]:
myHandle=ql.YieldTermStructureHandle(dcurve)

In [25]:
ql.CashFlows.npv(cfs,spotCurveHandle,True,ql.Date(31,10,1929))

191.97348605998428